# 深度学习分享——从线性回归开始

本次分享从简单的线性回归开始，借由该过程为大家介绍一个最简单的网络应如何搭建。

实际上，所有的网络训练也都是同样的套路，不同的是网络内部的结构和优化方法。

## 机器学习的基本过程

机器学习的本质，就是针对一个问题，找到一个模式，通过不断的训练自动的发现这个模式的规律。使得将来出现此类的数据，都可以预测出结果。

**例如：**

* 假设我们并不了解天文学，仅凭借直觉的方式认为，白天和黑夜的区别可能与天上有没有太阳相关
* 于是我们提出了一个猜想： 白天 = 太阳在天上\*a   (其中 a 可以等于 0 或 1 ）
* 通过不断的总结规律与实验发现：白天时，太阳都在天上
* 某一天，当我们走出家门看到太阳在天上时，我们可以大胆的推测，这是一个白天！

整个过程可以被抽象描述为：

1. 输入训练数据
2. 利用特定的机器学习方法建立估计函数
3. 得到函数后向这一模型输入测试数据
4. 函数有能力对没有见过的数据进行正确估计
<img src="images/ML_process.png" style="width:300px;height:180px;">

##  如何让我们的网络学习一种映射


给定一组数据 $<X,Y>$，求解出函数 $f$，使得 $y=f(x)$，其中$X$,$Y$均为张量。最终网络可以依据输入$x$，准确预测出$y_{\_predict}$。

本次我们用很少的数据来学习一个很简单的关系：

- 输入 X=[1 2 3 4]
- 输出 Y=[2,4,6,8]

在fluid中，我们可以这样定义数据：


In [ ]:
import paddle.fluid as fluid
import numpy


#定义X数值
train_data=numpy.array([[1.0],[2.0],[3.0],[4.0]]).astype('float32')
#定义期望预测的真实值y_true
y_true = numpy.array([[2.0],[4.0],[6.0],[8.0]]).astype('float32')

##  线性回归的基本概念 

肉眼可见，上述一组数据是 Y=2\*X 的关系，即 Y 与 X 呈线性关系。因此这是一个非常简单的**线性回归**模型。

线性回归是机器学习中最简单也是最重要的模型之一，其模型建立同样遵循上图流程：获取数据、数据预处理、训练模型、应用模型。

回归模型可以理解为：存在一个点集，用一条曲线去拟合它分布的过程。如果拟合曲线是一条直线，则称为线性回归。如果是一条二次曲线，则被称为二次回归。线性回归是回归模型中最简单的一种。

在线性回归中有几个基本的概念需要掌握：
- 假设函数（Hypothesis Function）
- 损失函数（Loss Function）
- 优化算法（Optimization Algorithm）

### 1. 假设函数：

假设函数是指，用数学的方法描述自变量和因变量之间的关系，它们之间可以是一个线性函数或非线性函数。
在本次模型中，我们的假设函数为 $ \hat{Y}= aX_0+b $ ，其中，$\hat{Y}$表示模型的预测结果，用来和真实的Y区分。模型要学习的参数即：a,b。

** 配置网络结构 **


线性回归的模型其实就是一个采用线性激活函数（linear activation）的全连接层（fully-connected layer，fc_layer），因此在fluid中利用全连接层模型构造线性回归，这样一个全连接层就可以看做是一个简单的神经网络，只包含输入层和输出层即可。本次的模型由于只有一个影响参数，因此输入只含一个$X_0$。

<img src="images/FC.png" style="width:380px;height:180px;">

接下来就让我们利用框架提供的接口，搭建我们自己的网络吧！


In [ ]:
#定义输入数据类型
x = fluid.layers.data(name="x",shape=[1],dtype='float32')
#搭建全连接网络
y_predict = fluid.layers.fc(input=x,size=1,act=None)

这样的网络就可以进行预测了，虽然输出结果只是一组随机数，离预期结果仍相差甚远，但我们的网络已经可以跑通了

添加一些框架所需的必要步骤，开始我们的预测吧：

In [ ]:
#加载库
import paddle.fluid as fluid
import numpy
#定义数据
train_data=numpy.array([[1.0],[2.0],[3.0],[4.0]]).astype('float32')
y_true = numpy.array([[2.0],[4.0],[6.0],[8.0]]).astype('float32')
#定义预测函数
x = fluid.layers.data(name="x",shape=[1],dtype='float32')
y = fluid.layers.data(name="y",shape=[1],dtype='float32')
y_predict = fluid.layers.fc(name='y_predict',input=x,size=1,act=None)
#参数初始化
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())
#开始训练
outs = exe.run(
    feed={'x':train_data},
    fetch_list=[y_predict.name])
#观察结果
print outs

如何让我们的网络学习的更加精确呢？这就需要加入一些优化的策略，其中一个很关键的概念，叫损失函数。

### 2. 损失函数：

完成模型搭建后，如何评估预测结果的好坏呢？我们通常在设计的网络中添加损失函数，以计算真实值与预测值的差。

对于线性模型来讲，最常用的损失函数就是均方误差（Mean Squared Error， MSE）。
$$MSE=\frac{1}{n}\sum_{i=1}^{n}(\hat{Y_i}-Y_i)^2$$
即对于一个大小为n的测试集，MSE是n个数据预测结果误差平方的均值。

fluid中有写好的API接口，我们可以直接调用接口计算损失函数的值

    cost = fluid.layers.square_error_cost(input=y_predict, label=y_true)
    avg_cost = fluid.layers.mean(cost)


输出结果看一下：

In [ ]:
#加载库
import paddle.fluid as fluid
import numpy
#定义数据
train_data=numpy.array([[1.0],[2.0],[3.0],[4.0]]).astype('float32')
y_true = numpy.array([[2.0],[4.0],[6.0],[8.0]]).astype('float32')
#定义网络
x = fluid.layers.data(name="x",shape=[1],dtype='float32')
y = fluid.layers.data(name="y",shape=[1],dtype='float32')
y_predict = fluid.layers.fc(input=x,size=1,act=None)
#定义损失函数
cost = fluid.layers.square_error_cost(input=y_predict,label=y)
avg_cost = fluid.layers.mean(cost)
#参数初始化
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())
#开始训练
outs = exe.run(
    feed={'x':train_data,'y':y_true},
    fetch_list=[y_predict.name,avg_cost.name])
#观察结果
print outs

可以看到第一轮计算后的损失函数很大，仍有很大的下降空间。

### 3. 优化算法：

在模型训练中优化算法也是至关重要的，它决定了一个模型的精度和运算速度。本章的线性回归实例中主要使用了梯度下降法进行优化。

**梯度下降**

是深度学习中非常重要的概念，值得庆幸的是它也十分容易理解。损失函数$J(w,b)$可以理解为变量$w$和$b$的函数。观察下图，垂直轴表示损失函数的值，两个水平轴分别表示变量$w$和$b$。实际上，可能是更高维的向量，但是为了方便说明，在这里假设$w$和$b$都是一个实数。算法的最终目标是找到损失函数的最小值。而这个寻找过程就是不断地微调变量$w$和$b$的值，一步一步地试出这个最小值。而试的方法就是沿着梯度方向逐步移动。本例中让图中的圆点表示损失函数的某个值，那么梯度下降就是让圆点沿着曲面下降，直到取到最小值或逼近最小值。

因为是凸函数，所以无论初始化在曲面上的哪一点，最终都会收敛到同一点或者相近的点。

<img src="images/gradient_descent.png" style="width:380px;height:180px;">

确定损失函数后，可以通过前向计算得到损失值，然后通过链式求导法则得到参数的梯度值。



获取梯度值后需要更新参数，最简单的算法是随机梯度下降法：w=w−η⋅g，fluid.optimizer.SGD可以实现这一过程：

    sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.01)
    

让我们的网络训练100次，查看结果： 

In [34]:
#加载库
import paddle.fluid as fluid
import numpy


#定义数据
train_data=numpy.array([[1.0],[2.0],[3.0],[4.0]]).astype('float32')
y_true = numpy.array([[2.0],[4.0],[6.0],[8.0]]).astype('float32')


#定义网络
x = fluid.layers.data(name="x",shape=[1],dtype='float32')
y = fluid.layers.data(name="y",shape=[1],dtype='float32')
y_predict = fluid.layers.fc(input=x,size=1,act=None)


#定义损失函数
cost = fluid.layers.square_error_cost(input=y_predict,label=y)
avg_cost = fluid.layers.mean(cost)


#定义优化方法
sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.1)
sgd_optimizer.minimize(avg_cost)


#参数初始化
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())


##开始训练，迭代100次
for i in range(100):
    outs = exe.run(
        feed={'x':train_data,'y':y_true},
        fetch_list=[y_predict.name,avg_cost.name])

#观察结果
print outs

[array([[2.0355666],
       [4.0172343],
       [5.9989023],
       [7.98057  ]], dtype=float32), array([0.00048518], dtype=float32)]


可以看到100次迭代后，预测值已经非常接近真实值了，损失值也从一个很大的初始值下降到了0.001以下。 

通过100次迭代，我们的网络学习到了一个2倍的关系式~ 开心！

#### 打印参数验证是否真的是2倍的关系呢？

In [29]:
#print fluid.default_main_program().to_string(True)

In [35]:
w = numpy.array(fluid.global_scope().find_var('fc_0.w_0').get_tensor())
b = numpy.array(fluid.global_scope().find_var('fc_0.b_0').get_tensor())
print w,b

[[1.9875268]] [0.03667285]


** 扩展 **


手写 反向传播+梯度下降，模拟机器训练过程

